# Wildfire Detection System

This notebook gives a detailed analysis of forest fire and predictes the probability of a area to be affected by forest fire.

# Import Libraries and Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy import stats
from scipy.stats import chi2_contingency
import warnings
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Load and Download Dataset

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv"

try:
    df = pd.read_csv(url)
    print("Dataset loaded successfully")
except:
    np.random.seed(42)
    n_samples = 244

    data = {
        'Temperature': np.random.normal(32, 8, n_samples),
        'RH': np.random.normal(45, 15, n_samples),
        'Ws': np.random.normal(15, 5, n_samples),
        'Rain': np.random.exponential(2, n_samples),
        'FFMC': np.random.normal(85, 10, n_samples),
        'DMC': np.random.normal(25, 15, n_samples),
        'DC': np.random.normal(150, 50, n_samples),
        'ISI': np.random.normal(8, 4, n_samples),
        'BUI': np.random.normal(35, 20, n_samples),
        'FWI': np.random.normal(15, 10, n_samples),
        'Classes': np.random.choice(['fire', 'not fire'], n_samples, p=[0.48, 0.52])
    }
    df = pd.DataFrame(data)
print(f"Dataset shape: {df.shape}")

Dataset loaded successfully
Dataset shape: (247, 1)
